## Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")

from implicit.nearest_neighbours import BM25Recommender
from rectools.dataset import Dataset
from rectools import Columns
from rectools.model_selection import TimeRangeSplit
import pandas as pd


# from service.models.userknn import UserKnn
from service.models.userknn import UserKnn
from service.utils import save_model, load_model

## Data

In [ ]:
!mkdir ../data
!wget https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip -O ../data/data_original.zip
!unzip ../data/data_original.zip -d ../data

In [2]:
interactions = pd.read_csv('../data/kion_train/interactions.csv')
users = pd.read_csv('../data/kion_train/users.csv')
items = pd.read_csv('../data/kion_train/items.csv')


# rename columns, convert timestamp
interactions.rename(columns={'last_watch_dt': Columns.Datetime,
                            'total_dur': Columns.Weight},
                    inplace=True)

interactions['datetime'] = pd.to_datetime(interactions['datetime'])


## Train test split

In [3]:
n_folds = 1
unit = "W"
n_units = 1
periods = n_folds + 1

last_date = interactions[Columns.Datetime].max().normalize()
start_date = last_date - pd.Timedelta(n_folds * n_units + 1, unit=unit)  # TimeDelta возвращает длительность промежутка между датами
print(f"Start date and last date of the test fold: {start_date, last_date}")

date_range = pd.date_range(start=start_date, periods=periods, freq=unit, tz=last_date.tz)
print(f"Test fold borders: {date_range.values.astype('datetime64[D]')}")

# generator of folds
cv = TimeRangeSplit(
    date_range=date_range,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)
print(f"Real number of folds: {cv.get_n_splits(interactions)}")

Start date and last date of the test fold: (Timestamp('2021-08-08 00:00:00'), Timestamp('2021-08-22 00:00:00'))
Test fold borders: ['2021-08-08' '2021-08-15']
Real number of folds: 1


In [4]:
(train_ids, test_ids, fold_info) = cv.split(interactions, collect_fold_stats=True).__next__()

In [5]:
train = interactions.loc[train_ids].reset_index(drop=True)
test = interactions.loc[test_ids].reset_index(drop=True)

## Train Model

In [6]:
model = UserKnn(dist_model=BM25Recommender(K=10, K1=2), n_neighbors=10)

In [7]:
model.fit(train)

  0%|          | 0/842129 [00:00<?, ?it/s]

In [8]:
recs = pd.DataFrame({'user_id': [964868]})
model.predict(recs=recs, n_recs=10)

[9506, 13058, 5755, 7582, 5569, 11661]

In [9]:
model.predict_user(user_id=964868)

[9506, 13058, 5755, 7582, 5569, 11661]

In [10]:
save_model(model_path='../models/userknn.pickle', model=model)
reco_model = load_model(model_path='../models/userknn.pickle')
reco_model.predict(recs=recs, n_recs=10)

[9506, 13058, 5755, 7582, 5569, 11661]